# 工具变量的定义

$该变量主要解决内生性问题，与随机扰动项不相关,即\rho(x_{工具},\varepsilon)=0,\\
与内生变量相关,即\rho(x_{工具}，x_{内生}) \neq 0 $

# 什么是二阶段最小二乘IV2SLS
$本质:把内生变量拆成两个东西，一部分与随机扰动项不相关，\\
另一部分与随机扰动项相关$

$该模型分为两个阶段的回归\\
第一阶段：通过工具变量，拟合生成不与随机扰动项相关的数据x_{fit}\\
第二阶段：x_{fit}再与被解释变量做回归(避免了内生性问题)$

# 伍德里奇参考案例
研究美国女性教育回报问题

## 研究背景

$研究美国女性的受教育时间与她们的工资收入之间的关系，\\
在此基础上加入外生变量：工作经验,以及它的平方项\\
即log(wage) = \beta_0 +\beta_1educ+\beta_2exper+\beta_3exper^2+u$

$注:加入工具变量,因为在内生变量中只考虑了受教育时间，\\
大概率会存在遗漏变量，从而引发内生性问题，因此考虑将父亲的\\
受教育程度fatheduc、母亲的受教育程度motheduc作为工具变量$

In [26]:
# 导入数据
import wooldridge as wool
import pandas as pd
data=wool.dataWoo('mroz')
#检查数据是否有缺失值
print(data.isnull().sum()[data.isnull().sum()!=0])
#即lwage有缺失值，删除含有确实值的样本
data.dropna(subset=['lwage'],inplace=True)
data.head()

wage     325
lwage    325
dtype: int64


,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310.0,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800.0,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040.0,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300.0,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300.0,0.6215,12,14,9.5,1,7,20.100058,1.524272,49


## 第一种方法：statsmodel进行2SLS回归
需要进行两次回归

### 第一阶段:首先将内生变量与工具变量回归，生成x_fit

$\ educ = \beta_0+\beta_1exper+\beta_2exper^2+\beta_3fathereduc+\beta_4fathereduc+\varepsilon$

In [29]:
import statsmodels.formula.api as sm
reg_first=sm.ols(formula='educ~exper+expersq+fatheduc+motheduc',
                data=data)
result_first=reg_first.fit()
data['educ_fit']=result_first.fittedvalues#这里生成了x拟合值
result_first.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   educ   R-squared:                       0.211
Model:                            OLS   Adj. R-squared:                  0.204
Method:                 Least Squares   F-statistic:                     28.36
Date:                Tue, 15 Nov 2022   Prob (F-statistic):           6.87e-21
Time:                        09:22:28   Log-Likelihood:                -909.72
No. Observations:                 428   AIC:                             1829.
Df Residuals:                     423   BIC:                             1850.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.1026      0.427     21.340      0.000       8.264       9.941
exper          0.0452      0.040      1.124      0.262      -0.034       0.124
expersq       -0.0010      0.001     -0.839      0.402      -0.003       0.001
fatheduc       0.1895      0.034      5.615      0.000       0.123       0.256
motheduc       0.1576      0.036      4.391      0.000       0.087       0.228
==============================================================================
Omnibus:                       10.903   Durbin-Watson:                   1.940
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               20.371
Skew:                          -0.013   Prob(JB):                     3.77e-05
Kurtosis:                       4.068   Cond. No.                     1.55e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.55e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 第二阶段：将生成的x_fit值与被解释变量做回归
这个才是我们需要真正分析的

$log(wage)=\alpha_0+\alpha_1educ_{fit}+\alpha_2exper+\alpha_3expersq+u$

In [31]:
reg_second=sm.ols(formula='lwage~educ_fit+exper+expersq',
                 data=data)
results_second = reg_second.fit()
results_second.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     7.405
Date:                Tue, 15 Nov 2022   Prob (F-statistic):           7.62e-05
Time:                        09:29:22   Log-Likelihood:                -457.17
No. Observations:                 428   AIC:                             922.3
Df Residuals:                     424   BIC:                             938.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0481      0.420      0.115      0.909      -0.777       0.873
educ_fit       0.0614      0.033      1.863      0.063      -0.003       0.126
exper          0.0442      0.014      3.136      0.002       0.016       0.072
expersq       -0.0009      0.000     -2.134      0.033      -0.002   -7.11e-05
==============================================================================
Omnibus:                       53.587   Durbin-Watson:                   1.959
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              168.354
Skew:                          -0.551   Prob(JB):                     2.77e-37
Kurtosis:                       5.868   Cond. No.                     4.41e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.41e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## 第二种方法：linearmodel进行2SLS回归
相对第一种，代码实现跟简单

In [32]:
from linearmodels.iv import IV2SLS

$formula的基本形式:dep-exog+[endog~instr]\\
dep为被解释变量,exog为外生变量,endog为内生变量\\
instr为工具变量$

In [33]:
reg_iv = IV2SLS.from_formula(
        formula='lwage~1+exper+expersq+[educ~fatheduc+motheduc]',
        data=data)
result_iv=reg_iv.fit(cov_type='unadjusted',debiased=True)
result_iv

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                  lwage   R-squared:                      0.1357
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1296
No. Observations:                 428   F-statistic:                    8.1407
Date:                Tue, Nov 15 2022   P-value (F-stat)                0.0000
Time:                        09:36:57   Distribution:                 F(3,424)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      0.0481     0.4003     0.1202     0.9044     -0.7388      0.8350
exper          0.0442     0.0134     3.2883     0.0011      0.0178      0.0706
expersq       -0.0009     0.0004    -2.2380     0.0257     -0.0017     -0.0001
educ           0.0614     0.0314     1.9530     0.0515     -0.0004      0.1232
==============================================================================

Endogenous: educ
Instruments: fatheduc, motheduc
Unadjusted Covariance (Homoskedastic)
Debiased: True
IVResults, id: 0x149d50e5d60

# 检验工具变量、内生性
<span class="mark">这一步是关键，因为能够考察你是否工具变量选对了</span>

## 内生性检验
$刚才我们把x拆开了2部分，只用了x不与随机扰动项相关的那部分，\\
另外一部分与随机扰动项相关的部分就是用来检验的.$

In [36]:
#即在第一阶段中
reg_first=sm.ols(formula='educ~exper+expersq+fatheduc+motheduc',
                data=data)
result_first=reg_first.fit()
data['resid']=result_first.resid#与随机扰动项相关的部分
data['resid'].head()

0   -0.756017
1    0.266442
2   -0.771979
3    0.232317
4    0.085385
Name: resid, dtype: float64

$把生成的与随机扰动项相关的部分与被解释变量做回归,\\
如果这部分的方程系数显著，说明了拆出来的这部分(与随机扰动项相关)\\
与被解释变量相关，即最开始的内生变量educ是具有内生性的。$

In [37]:
reg_second=sm.ols(formula='lwage~resid+educ+exper+expersq',
                 data=data)
result_second=reg_second.fit()
result_second.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     20.50
Date:                Tue, 15 Nov 2022   Prob (F-statistic):           1.89e-15
Time:                        09:49:59   Log-Likelihood:                -430.19
No. Observations:                 428   AIC:                             870.4
Df Residuals:                     423   BIC:                             890.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0481      0.395      0.122      0.903      -0.727       0.824
resid          0.0582      0.035      1.671      0.095      -0.010       0.127
educ           0.0614      0.031      1.981      0.048       0.000       0.122
exper          0.0442      0.013      3.336      0.001       0.018       0.070
expersq       -0.0009      0.000     -2.271      0.024      -0.002      -0.000
==============================================================================
Omnibus:                       74.968   Durbin-Watson:                   1.931
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              278.059
Skew:                          -0.736   Prob(JB):                     4.17e-61
Kurtosis:                       6.664   Cond. No.                     4.42e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.42e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## 工具变量与随机扰动项是否相关

### 通过刚才第二个方法获得残差
$log(wage) = \beta_0 +\beta_1educ+\beta_2exper+\beta_3exper^2+\beta_4fatheduc+\beta_5mothereduc+u$

In [38]:
reg_iv = IV2SLS.from_formula(
        formula='lwage~1+exper+expersq+[educ~fatheduc+motheduc]',
        data=data)
result_iv=reg_iv.fit(cov_type='unadjusted',debiased=True)
data['resid_iv']=result_iv.resids#获得残差

### 将残差与所有外生变量和工具变量回归
判断工具变量的系数是否显著

In [39]:
reg_judge=sm.ols(formula='resid_iv~exper+expersq+fatheduc+motheduc',
                data=data)
result_judge=reg_judge.fit()

### p值判断
$H0:工具变量与随机扰动项不相关.\\
nR^2 \sim X_q^2$

In [44]:
import scipy.stats as stat
R2=result_judge.rsquared#可决系数
n=result_judge.nobs# 样本数量
q=2-1#工具变量数目-内生变量数目
nR2=n*R2
p=1-stat.chi2.cdf(nR2,q)#卡方
print(f'p值为{p}')
if p>0.05:
    print('接受原假设，工具变量与随机扰动项不相关,工具变量适用')
else:
    print('拒绝原假设，工具变量与随机扰动项相关')

p值为0.538637233071513
接受原假设，工具变量与随机扰动项不相关,工具变量适用
